In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime as DT
import pickle
print('system: ', sys.version)
print('numpy: ', np.__version__)
print('panda: ', pd.__version__)

system:  3.5.1 |Anaconda 2.4.1 (64-bit)| (default, Dec  7 2015, 15:00:12) [MSC v.1900 64 bit (AMD64)]
numpy:  1.10.1
panda:  0.17.1


In [2]:
# Create Trend Ranges, based on visual inspection of previous graph, and add Trends to Order dataframe
trend_range = [0, 6, 33, 60, 77, 86, 150, 171, 207, 222, 271, 314, 331, 348]
trend_labels = ['UP1', 'FLAT1', 'DOWN1', 'UP2', 'DOWN2', 'FLAT2', 'DOWN3', 'UP3', 'DOWN4', 'UP4', 
               'DOWN5', 'UP6', 'FLAT3']

In [3]:
with open('df_Ind.pickle', 'rb') as file:
    df_Ind = pickle.load(file)
    
with open('df_ord.pickle', 'rb') as file:
    df_ord = pickle.load(file)

In [5]:
#define Gain and Loss filters
Gain = (df_ord['Profit'] > 0)
Loss = (df_ord['Profit'] < 0)

#define filters for trends
up1 = (['Trend'] == 'UP1')
flat1 = (['Trend'] == 'FLAT1')
down1 = (['Trend'] == 'DOWN1')
up2 = (['Trend'] == 'UP2')
down2 = (['Trend'] == 'DOWN2')
flat2 = (['Trend'] == 'FLAT2')
down3 = (['Trend'] == 'DOWN3')
up3 = (['Trend'] == 'UP3')
down4 = (['Trend'] == 'DOWN4')
up4 = (['Trend'] == 'UP4')
down5 = (['Trend'] == 'DOWN5')
up6 = (['Trend'] == 'UP6')
flat3 = (['Trend'] == 'FLAT3')

In [6]:
# Pull Gain orders from Order dataframe
gain = df_ord.loc[Gain, ('DateTime', 'Trend', 'Ticket', 'Profit', 'Type')]
loss = df_ord.loc[Loss, ('DateTime', 'Trend', 'Ticket', 'Profit', 'Type')]

In [7]:
df_Ind.head()

,DateTime,Date,Time,Ticket,Open/Close,BBAND(10),BBAND(20),BBAND(30),BBAND(40),BBAND(50),...,MACD(14) MAIN,MACD(14) SIGNAL,MACD(16) MAIN,MACD(16) SIGNAL,MACD(18) MAIN,MACD(18) SIGNAL,MACD(20) MAIN,MACD(20) SIGNAL,DateTime_sec,Trend
0,2015-01-02 08:00:00,2015-01-02,08:00:00,-1,ON_NewBar,1.209722,1.210053,1.210167,1.210231,1.210213,...,-0.000425,-0.000047,-0.000425,-0.000036,-0.000425,-0.000026,-0.000425,-0.000016,1420185600,NaN
1,2015-01-02 08:01:00,2015-01-02,08:01:00,-1,ON_NewBar,1.209193,1.209771,1.209984,1.210093,1.210109,...,-0.000757,-0.000105,-0.000757,-0.000088,-0.000757,-0.000073,-0.000757,-0.000059,1420185660,NaN
2,2015-01-02 08:02:00,2015-01-02,08:02:00,-1,ON_NewBar,1.208657,1.209489,1.209802,1.209957,1.210005,...,-0.001009,-0.000180,-0.001009,-0.000154,-0.001009,-0.000132,-0.001009,-0.000113,1420185720,NaN
3,2015-01-02 08:03:00,2015-01-02,08:03:00,-1,ON_NewBar,1.208082,1.209193,1.209604,1.209807,1.209895,...,-0.001229,-0.000271,-0.001229,-0.000234,-0.001229,-0.000204,-0.001229,-0.000179,1420185780,NaN
4,2015-01-02 08:04:00,2015-01-02,08:04:00,-1,ON_NewBar,1.207358,1.208821,1.209358,1.209620,1.209755,...,-0.001498,-0.000383,-0.001498,-0.000334,-0.001498,-0.000294,-0.001498,-0.000260,1420185840,NaN


In [19]:
rolling_window = 7
df_Ind['bband100_std'] = pd.rolling_std(df_Ind['BBAND(100)'], rolling_window)
df_Ind['macd12main_std'] = pd.rolling_std(df_Ind['MACD(12) MAIN'], rolling_window)
df_Ind['macd2signal_std'] = pd.rolling_std(df_Ind['MACD(2) SIGNAL'], rolling_window)
df_Ind['macd10signal_std'] = pd.rolling_std(df_Ind['MACD(10) SIGNAL'], rolling_window)
df_Ind['macd12signal_std'] = pd.rolling_std(df_Ind['MACD(12) SIGNAL'], rolling_window)
df_Ind['macd20signal_std'] = pd.rolling_std(df_Ind['MACD(20) SIGNAL'], rolling_window)

In [33]:
gain_merged = pd.merge(gain, df_Ind[['DateTime', 'BBAND(100)', 'MACD(12) MAIN', 'MACD(10) SIGNAL',
                                'bband100_std', 'macd12main_std', 'macd10signal_std']], on='DateTime', how='inner')

loss_merged = pd.merge(loss, df_Ind[['DateTime', 'BBAND(100)', 'MACD(12) MAIN', 'MACD(10) SIGNAL',
                                'bband100_std', 'macd12main_std', 'macd10signal_std']], on='DateTime', how='inner')

In [35]:
gain_summary = gain_merged.groupby(['Trend']).mean()[['bband100_std', 'macd12main_std', 'macd10signal_std']]
loss_summary = loss_merged.groupby(['Trend']).mean()[['bband100_std', 'macd12main_std', 'macd10signal_std']]

In [47]:
gain_summary

,bband100_std,macd12main_std,macd10signal_std
Trend,,,
DOWN1,0.000038,0.000072,0.000042
DOWN2,0.000035,0.000081,0.000067
DOWN3,0.000037,0.000082,0.000044
DOWN4,0.000034,0.000111,0.000052
DOWN5,0.000073,0.000111,0.000081
FLAT1,0.000068,0.000127,0.000088
FLAT2,0.000057,0.000118,0.000068
FLAT3,0.000100,0.000084,0.000071
UP1,0.000063,0.000069,0.000037


In [69]:
constant_df = pd.DataFrame ({'Trend': ['DOWN1', 'DOWN2', 'DOWN3', 'DOWN4', 'DOWN5', 'FLAT1', 'FLAT2', 'FLAT3', 'UP1',
               'UP2', 'UP3', 'UP4', 'UP6']})

#Gain Summary Chart
constant_df['const_val1'] = .00006
constant_df['const_val2'] = .000075

gain_target = pd.merge(gain_summary.reset_index(), constant_df, how='outer')
ax_gain = gain_target[['Trend', 'const_val1', 'const_val2']].plot(x='Trend', linestyle='-', marker=None, color='grey')
gain_target[['Trend', 'bband100_std', 'macd12main_std', 'macd10signal_std']].plot(x='Trend', kind='bar', ax=ax_gain)
plt.title('Gain Summary')


#Loss Summary chart
constant_df['const_val1'] = .00002
loss_target = pd.merge(loss_summary.reset_index(), constant_df, how='outer')
ax_loss = loss_target[['Trend', 'const_val1']].plot(x='Trend', linestyle='-', marker=None, color='grey')
loss_target[['Trend', 'bband100_std', 'macd12main_std', 'macd10signal_std']].plot(x='Trend', kind='bar', ax=ax_loss)
plt.title('Loss Summary')

plt.show()


In [70]:
gain_summary.describe()

,bband100_std,macd12main_std,macd10signal_std
count,13.000000,13.000000,13.000000
mean,0.000054,0.000092,0.000056
std,0.000019,0.000019,0.000018
min,0.000034,0.000069,0.000029
25%,0.000038,0.000077,0.000042
50%,0.000055,0.000084,0.000052
75%,0.000063,0.000111,0.000068
max,0.000100,0.000127,0.000088


In [71]:
loss_summary.describe()

,bband100_std,macd12main_std,macd10signal_std
count,13.000000,13.000000,13.000000
mean,0.000049,0.000092,0.000046
std,0.000014,0.000020,0.000015
min,0.000025,0.000063,0.000029
25%,0.000043,0.000075,0.000035
50%,0.000050,0.000099,0.000041
75%,0.000054,0.000107,0.000050
max,0.000075,0.000126,0.000077


In [84]:
len(gain_merged[(gain_merged['macd10signal_std'] > 0.000068)])

55

In [83]:
len(loss_merged[(loss_merged['macd10signal_std'] > 0.000068)])

24